---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [193]:
import pandas as pd
import re

doc = []
with open('../readonly/dates.txt') as file:
    for line in file:
        doc.append(line.strip())

df = pd.DataFrame(doc, columns=['text'])
#df.head(10)


In [194]:
def extract(data):
    
    mmddyyyy = re.findall(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{4}\b', data)
    mmddyy = re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2}\b', data)
    mmyyyy = re.findall(r'\d{1,2}[/]\d{4}\b', data)
    mon_dd_yyyy = re.findall('(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.]?(?: \d{1,2}[a-z]*)[,]? \d{4}', data)
    dd_mon_yyyy = re.findall('(?:\d{1,2} )(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]? \d{4}', data)
    mon_year = re.findall(r'(?<![0-9] )(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]? \d{4}', data)
    year = re.findall(r'(?<![1-9])\d{4}(?![1-9])', data)
  
    if mmddyyyy:
        return mmddyyyy[0] #+ ': mmddyyy'
    elif mmddyy:
        return mmddyy[0] #+ ': mmddyy'
    elif mmyyyy:
        s = mmyyyy[0].split('/')
        return s[0] + '/1/' + s[1] #+ ': mmyyyy'
    elif mon_dd_yyyy:
        return mon_dd_yyyy[0] #+ ': mon_dd_yyyy'
    elif dd_mon_yyyy:
        return dd_mon_yyyy[0] #+ ': dd_mon_yyyy'
    elif mon_year:
        s = mon_year[0].split()
        s[0] = s[0][:3]
        s.append(s[1])
        s[1] = '1,'
        return ' '.join(s) #+ ': mon_year'
    elif year:
        return 'January 1, ' + year[0] #+ ': year'
    else:
        return None

df['date'] = df['text'].apply(extract)
df['date-time'] = pd.to_datetime(df['date'])
df.sort_values('date-time', inplace=True)
pd.Series(df.index.tolist()).astype('int32')

df.to_html("debug.html")
#df['date1'].value_counts(dropna=False)
#df[df['date1'].isna()].index

In [7]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here